In [47]:
import requests
import time,datetime,pytz
import pymysql as sql
import schedule

In [9]:
uid='13354765'

url='https://api.bilibili.com/x/relation/stat?vmid='+uid

In [13]:
r = requests.get(url).json()

follower=r['data']['follower']

In [29]:
def currentTimeStamp():
    return int(time.mktime(datetime.datetime.now().timetuple()))

def timeStampToTime():
    return datetime.datetime.fromtimestamp(currentTimeStamp(),pytz.timezone('Asia/Shanghai'))

In [31]:
(follower,currentTimeStamp())

(1735919, 1554359297)

In [32]:
with open('passwd.txt','r') as f:
    mysql_user=f.readline()[:-1]
    mysql_passwd=f.readline()[:-1]
    mysql_database=f.readline()[:-1]

In [34]:
conn = sql.connect(user=mysql_user,passwd=mysql_passwd,db=mysql_database,charset='utf8')
cur = conn.cursor()

In [41]:
cur.execute('show databases;')
cur.fetchall()

(('bilibili',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('user13354765',),
 ('user13354765test',),
 ('user13571473',),
 ('user28496477',),
 ('user383444910',),
 ('user946974test',))

In [40]:
cur.execute('create database user{}test;'.format(uid))
cur.fetchall()

()

In [42]:
cur.execute('use user13354765test;')
cur.fetchall()

()

In [43]:
cur.execute('''
create table userinfo(
id int unsigned primary key auto_increment,
follower int unsigned not null default 0,
timestamp int unsigned not null default 0
)charset utf8;
''')
cur.fetchall()

()

In [44]:
cur.execute('show tables;')
cur.fetchall()

(('userinfo',),)

In [45]:
cur.execute('insert into userinfo (follower,timestamp) values({},{});'.format(follower,currentTimeStamp()))
cur.fetchall()

()

In [52]:
cur.execute('select follower,timestamp from userinfo;'.format(follower,currentTimeStamp()))
cur.fetchall()

((1735919, 1554359846), (1735957, 1554360031), (1735957, 1554360044))

In [48]:
def saveFollowerData(conn):
    r = requests.get(url).json()
    follower=r['data']['follower']
    cur.execute('insert into userinfo (follower,timestamp) values({},{});'.format(follower,currentTimeStamp()))
    cur.fetchall()

In [51]:
saveFollowerData(conn)

In [53]:
schedule.every().days.at('00:00').do(saveFollowerData,conn)
schedule.every().days.at('06:00').do(saveFollowerData,conn)
schedule.every().days.at('12:00').do(saveFollowerData,conn)
schedule.every().days.at('18:00').do(saveFollowerData,conn)

Every 1 day at 18:00:00 do saveFollowerData(<pymysql.connections.Connection object at 0x7f78f78b6e10>) (last run: [never], next run: 2019-04-04 18:00:00)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)